In [1]:
from typing import Any, Optional

import numpy as np
import scipy as sp
import sympy as sm
import pandas as pd

import bottleneck as bn
import numexpr as ne
import numba as nb

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tools import add_constant

sns.set()

In [2]:
import statsmodels as sms
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [3]:
x, y = make_classification(10000, n_features=4, n_classes=2, n_informative=4,
                           n_repeated=0, n_redundant=0, flip_y=0.2, random_state=1)
x = pd.DataFrame(x)
x

,0,1,2,3
0,0.961191,0.097541,-0.138425,0.057640
1,2.351658,-0.102018,0.102338,2.886535
2,2.707095,-0.989234,-2.414672,1.074636
3,0.300021,-1.392938,1.836359,1.425227
4,0.196385,-0.006582,2.100452,-0.451834
...,...,...,...,...
9995,1.447341,0.694426,0.022308,0.207010
9996,3.571888,-1.047510,-1.785897,0.065202
9997,1.835292,-1.083098,-0.179020,0.922715
9998,-2.216549,0.734841,3.438741,2.103342


In [4]:
#!pip install dtale


In [5]:
x1, x2, y1, y2 = train_test_split(x, y, test_size=0.2, random_state=0)

In [6]:
model = LogisticRegression(class_weight=[1, 50])
model.fit(x1, y1)
y_model = model.predict_proba(x2)[:, 1]

In [7]:
roc_auc_score(y2, y_model) * 2 - 1

0.7065799586876111

In [8]:
df = pd.DataFrame({'y_predict': y_model, 'y_true': y2})
df

,y_predict,y_true
0,0.170240,1
1,0.207980,0
2,0.946901,0
3,0.946272,1
4,0.261681,0
...,...,...
1995,0.917215,1
1996,0.842191,1
1997,0.971491,1
1998,0.256967,1


In [9]:
df['y_predict'] = pd.qcut(df['y_predict'], 10, duplicates='drop')
df

,y_predict,y_true
0,"(0.124, 0.187]",1
1,"(0.187, 0.258]",0
2,"(0.926, 0.998]",0
3,"(0.926, 0.998]",1
4,"(0.258, 0.34]",0
...,...,...
1995,"(0.849, 0.926]",1
1996,"(0.727, 0.849]",1
1997,"(0.926, 0.998]",1
1998,"(0.187, 0.258]",1


In [10]:
df.groupby('y_predict')['y_true'].agg([np.mean, 'count'])


,mean,count
y_predict,,
"(0.013600000000000001, 0.124]",0.125,200
"(0.124, 0.187]",0.155,200
"(0.187, 0.258]",0.175,200
"(0.258, 0.34]",0.245,200
"(0.34, 0.443]",0.410,200
"(0.443, 0.588]",0.560,200
"(0.588, 0.727]",0.765,200
"(0.727, 0.849]",0.900,200
"(0.849, 0.926]",0.915,200


In [11]:
from statsmodels.discrete.discrete_model import Logit

In [12]:
model2 = Logit(y1, add_constant(x1)).fit()
print(roc_auc_score(y2, model2.predict(add_constant(x2))) * 2 - 1)
model2.summary2()

Optimization terminated successfully.
         Current function value: 0.494394
         Iterations 6
0.7065599429953084


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
================================================================
Model:              Logit            Pseudo R-squared: 0.287    
Dependent Variable: y                AIC:              7920.3103
Date:               2021-06-14 22:07 BIC:              7955.2463
No. Observations:   8000             Log-Likelihood:   -3955.2  
Df Model:           4                LL-Null:          -5544.8  
Df Residuals:       7995             LLR p-value:      0.0000   
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     6.0000                                      
------------------------------------------------------------------
         Coef.    Std.Err.      z       P>|z|     [0.025    0.975]
------------------------------------------------------------------
const    1.3127     0.0431    30.4719   0.0000    1.2283    1.3972
0       -1.0436     0.0307   -33.9682   0.0000   -1.1038   -0.9834
1        0.3049     0.0255    11.9757   0.0000    0.2550    0.3548
2       -1.1028     0.0286   -38.5973   0.0000   -1.1588   -1.0468
3       -0.3372     0.0235   -14.3222   0.0000   -0.3834   -0.2911
================================================================

"""

In [13]:
import scipy.stats as stats


class LogisticRegression2(LogisticRegression):
    """
    Wrapper Class for Logistic Regression which has the usual sklearn instance
    in an attribute self.model, and pvalues, z scores and estimated
    errors for each coefficient in

    self.z_scores
    self.p_values
    self.sigma_estimates

    as well as the negative hessian of the log Likelihood (Fisher information)

    self.F_ij
    """

    def __init__(self,
                 penalty: Any = 'l2',
                 *,
                 dual: Any = False,
                 tol: Any = 1e-4,
                 C: Any = 1.0,
                 fit_intercept: Any = True,
                 intercept_scaling: Any = 1,
                 class_weight: Any = None,
                 random_state: Any = None,
                 solver: Any = 'lbfgs',
                 max_iter: Any = 100,
                 multi_class: Any = 'auto',
                 verbose: Any = 0,
                 warm_start: Any = False,
                 n_jobs: Any = None,
                 l1_ratio: Any = None) -> Optional[Any]:
        super().__init__(penalty=penalty, dual=dual, tol=tol, C=C, fit_intercept=fit_intercept,
                                                  intercept_scaling=intercept_scaling, class_weight=class_weight,
                                                  random_state=random_state, solver=solver, max_iter=max_iter,
                                                  multi_class=multi_class, verbose=verbose, warm_start=warm_start,
                                                  n_jobs=n_jobs, l1_ratio=l1_ratio)

    def fit(self, X, y, **kwargs):
        super().fit(X, y)
        #### Get p-values for the fitted model ####
        denom = (2.0 * (1.0 + np.cosh(self.decision_function(X))))
        denom = np.tile(denom, (X.shape[1], 1)).T
        F_ij = np.dot((X / denom).T, X)  ## Fisher Information Matrix
        #add micro_value
        eps = 1e-6
        F_ij += np.eye(F_ij.shape[0]) * eps

        Cramer_Rao = np.linalg.inv(F_ij)  ## Inverse Information Matrix
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.coef_[0] / sigma_estimates  # z-score for eaach model coefficient
        p_values = [stats.norm.sf(abs(x)) * 2 for x in z_scores]  ### two tailed test for p-values

        self.z_scores = z_scores
        self.p_values = p_values
        self.sigma_estimates = sigma_estimates
        self.F_ij = F_ij
        alpha = 0.05
        q = stats.norm.ppf(1 - alpha / 2)
        lower = self.coef_[0] - q * sigma_estimates
        upper = self.coef_[0] + q * sigma_estimates
        self.conf_int_lower = lower
        self.conf_int_upper = upper
    def summary(self):
        return pd.DataFrame({'coef':self.coef_[0],'std err':self.sigma_estimates,'z':self.z_scores,
              'P>|z|':self.p_values,'[0.025':self.conf_int_lower,'0.975]':self.conf_int_upper})


In [14]:

model = LogisticRegression2(fit_intercept=False)
model.fit(add_constant(x1), y1)
#y_model=model.predict_proba(x2)[:,1]

In [15]:
model.summary()

,coef,std err,z,P>|z|,[0.025,0.975]
0,1.308445,0.043009,30.422262,2.789099e-203,1.224148,1.392742
1,-1.040962,0.030680,-33.929184,2.473807e-252,-1.101095,-0.980830
2,0.304628,0.025449,11.970005,5.102531e-33,0.254749,0.354508
3,-1.100208,0.028530,-38.563860,0.000000e+00,-1.156125,-1.044291
4,-0.336525,0.023528,-14.303061,2.093936e-46,-0.382640,-0.290411


In [16]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 8000
Model:                          Logit   Df Residuals:                     7995
Method:                           MLE   Df Model:                            4
Date:                Mon, 14 Jun 2021   Pseudo R-squ.:                  0.2867
Time:                        22:07:58   Log-Likelihood:                -3955.2
converged:                       True   LL-Null:                       -5544.8
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3127      0.043     30.472      0.000       1.228       1.397
0             -1.0436      0.031    -33.968      0.000      -1.104      -0.983
1              0.3049      0.025     11.976      0.000       0.255       0.355
2             -1.1028      0.029    -38.597      0.000      -1.159      -1.047
3             -0.3372      0.024    -14.322      0.000      -0.383      -0.291
==============================================================================
"""